In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import SimpleITK as sitk
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from medpy.io import load
from skimage.measure import block_reduce
import scipy
import os
import seaborn as sns
import nibabel as nb

# ResNet50 Accuracy

In [2]:
preprocess_input_resnet = tf.keras.applications.resnet.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [3]:
def normalize(arr):
    arr = arr + abs(np.amin(arr))
    assert np.amax(arr) != 0
    arr = arr / np.amax(arr)
    return arr

In [6]:
X = []
name = []

for i in range(500):
    try :
        t1 = sitk.ReadImage(os.getcwd() + "/Data/FA" + str(1000+i) + ".nii")
        t2 = sitk.GetArrayFromImage(t1)
        t3 = []
        for j in range(0,90):
            result = scipy.ndimage.zoom(t2[j], 224/288)
            t3.append(result)
        X.append(t3)
        
    except:
        continue
    
for i in range(500):
    try :
        t1 = sitk.ReadImage(os.getcwd() + "/Data/FA" + str(2000+i) + ".nii")
        t2 = sitk.GetArrayFromImage(t1)
        t3 = []
        for j in range(0,90):
            result = scipy.ndimage.zoom(t2[j], 224/288)
            t3.append(result)
        X.append(t3)
        
    except:
        continue

print(len(X))


64


2 3

In [7]:
X = np.array(X)
# 2
# (72, 40, 224, 224)

In [8]:
y = np.concatenate(( [0]*34, [1]*30 ))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=34)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(44, 90, 85, 71) (20, 90, 85, 71) (44,) (20,)


In [9]:
X_train = np.resize(X_train, ((40*44), 85, 71))
X_test = np.resize(X_test, ((40*20), 85, 71))

temp_y_train = np.array([])
for i in range(len(y_train)):
    for j in range(40):
        temp_y_train = np.append(temp_y_train, y_train[i])

temp_y_test = np.array([])
for i in range(len(y_test)):
    for j in range(40):
        temp_y_test = np.append(temp_y_test, y_test[i])

y_train = temp_y_train
y_test = temp_y_test

print(np.shape(X_train), np.shape(y_train), np.shape(X_test), np.shape(y_test))

(1760, 85, 71) (1760,) (800, 85, 71) (800,)


In [37]:
X_train_RGB = np.stack((X_train,)*3, axis=-1)

X_test_RGB = np.stack((X_test,)*3, axis=-1)

train_ds = preprocess_input_resnet(X_train_RGB)
test_ds = preprocess_input_resnet(X_test_RGB)

In [38]:
new_model = tf.keras.models.load_model('ResNet50/RESNET_ocd_resting_state_dataset_t1_2d.h5')
loss, acc = new_model.evaluate(test_ds, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

25/25 - 3s - loss: 0.6879 - accuracy: 0.5500 - 3s/epoch - 114ms/step
Restored model, accuracy: 55.00%


# Novel Accuracy

In [39]:
X = []
name = []

for i in range(500):
    try :
        t1 = sitk.ReadImage("./Data/FA" + str(1000+i) + ".nii")
        t2 = sitk.GetArrayFromImage(t1)
        t3 = []
        for j in range(0,90):
            result = scipy.ndimage.zoom(t2[j], 224/288)
            t3.append(result)
        X.append(t3)
        
    except:
        continue
    
for i in range(500):
    try :
        t1 = sitk.ReadImage("./Data/FA" + str(2000+i) + ".nii")
        t2 = sitk.GetArrayFromImage(t1)
        t3 = []
        for j in range(0,90):
            result = scipy.ndimage.zoom(t2[j], 224/288)
            t3.append(result)
        X.append(t3)
        
    except:
        continue

print(len(X))

64


In [40]:
X = np.array(X)

y = np.concatenate(( [0]*34, [1]*30 ))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=34)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(44, 90, 85, 71) (20, 90, 85, 71) (44,) (20,)


In [41]:
X_train = np.resize(X_train, ((40*44), 85, 71))
X_test = np.resize(X_test, ((40*20), 85, 71))

temp_y_train = np.array([])
for i in range(len(y_train)):
    for j in range(40):
        temp_y_train = np.append(temp_y_train, y_train[i])

temp_y_test = np.array([])
for i in range(len(y_test)):
    for j in range(40):
        temp_y_test = np.append(temp_y_test, y_test[i])

y_train = temp_y_train
y_test = temp_y_test

print(np.shape(X_train), np.shape(y_train), np.shape(X_test), np.shape(y_test))

(1760, 85, 71) (1760,) (800, 85, 71) (800,)


In [42]:
new_model = tf.keras.models.load_model('Novel/NOVEL_ocd_resting_state_dataset_t1_2d.h5')
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

25/25 - 0s - loss: 4.4628 - accuracy: 0.5050 - 305ms/epoch - 12ms/step
Restored model, accuracy: 50.50%
